In [1]:
import numpy as np

In [4]:
doc_word = np.load('science2k-doc-word.npy')

In [8]:
print doc_word

[[-0.2521619 -0.2521619  9.36371   ... -0.2521619 -0.2521619 -0.2521619]
 [-0.2875293 -0.2875293  8.229864  ... -0.2875293 -0.2875293 -0.2875293]
 [-0.3634041 -0.3634041  9.252468  ... -0.3634041 -0.3634041 -0.3634041]
 ...
 [10.04846   -0.7713402  9.132197  ... -0.7713402 -0.7713402 -0.7713402]
 [11.00702   -0.8423803 10.38288   ... -0.8423803 -0.8423803 -0.8423803]
 [ 9.710337  -0.7527946  9.556191  ... -0.7527946 -0.7527946 -0.7527946]]


In [20]:
print np.std(doc_word[56])

2.308974294870306
